# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [1]:
# 1) Seu código aqui - Ler o arquivo JSON salvo
import json
import pandas as pd

# Lê o arquivo
with open('pvl_dados.json', 'r', encoding='utf-8') as f:
    conteudo = f.read()

# Converte o texto JSON para dicionário
dados = json.loads(conteudo)

# Acessa a lista de itens e cria o DataFrame
df = pd.DataFrame(dados["items"])

# Exibe as 5 primeiras linhas
print(df.head())

   id_pleito tipo_interessado    interessado  cod_ibge  uf  \
0      67513           Estado           Pará        15  PA   
1      68629        Município  Pará de Minas   3147105  MG   
2      68631        Município          Jataí   5211909  GO   
3      68632        Município    Salesópolis   3545001  SP   
4      64864        Município     Dom Cavati   3122504  MG   

                num_pvl                                             status  \
0  PVL02.004298/2023-10  Encaminhado à PGFN com manifestação técnica fa...   
1  PVL02.000007/2024-97  Encaminhado à PGFN com manifestação técnica fa...   
2  PVL02.000709/2024-71  Encaminhado à PGFN com manifestação técnica fa...   
3  PVL02.000481/2024-19                         Em retificação pelo credor   
4  PVL02.001949/2023-10                                  Deferido (PVL-IF)   

           num_processo        data_protocolo  \
0  17944.105900/2023-90  2024-05-02T16:49:37Z   
1  17944.002500/2024-12  2024-06-11T13:32:55Z   
2  17944.00

In [2]:
# 2) Ver os três status mais frequentes
print(df["data_status"].value_counts().head(3))

data_status
02/07/2004    19
18/06/2012    15
10/06/2008    15
Name: count, dtype: int64


In [3]:
# 3) Nova coluna com o ano do status
df["ano_status"] = df["data_status"].str.slice(0, 4)
print(df[["data_status", "ano_status"]].head())

df["ano_status"] = df["data_status"].str.split("-").str[0]


  data_status ano_status
0  16/05/2024       16/0
1  28/06/2024       28/0
2  20/06/2024       20/0
3  14/03/2025       14/0
4  30/05/2023       30/0


In [4]:
# 4) Mostrar frequência do ano_status
print(df["ano_status"].value_counts())

ano_status
02/07/2004    19
18/06/2012    15
10/06/2008    15
02/10/2008    15
15/09/2008    14
              ..
06/09/2013     1
10/12/2021     1
13/04/2005     1
07/05/2025     1
25/03/2013     1
Name: count, Length: 2451, dtype: int64


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [16]:
# 1) Função que filtra UF e tipo de interessado:
filtro = (df["uf"] == "BA") & (df["tipo_interessado"] == "Estado")
df[filtro].to_csv("bahia_estado.csv", index=False)
print("Arquivo salvo como 'bahia_estado.csv'")

Arquivo salvo como 'bahia_estado.csv'


In [17]:
# 2) Quantas solicitações para o estado de Minas Gerais com status "Arquivado por descurto de prazo" estão registrados?:
filtro = (df["uf"] == "MG") & (df["status"] == "Arquivado por decurso de prazo")
quantidade = df[filtro].shape[0]
print("Total de solicitações arquivadas por decurso de prazo em MG:", quantidade)

Total de solicitações arquivadas por decurso de prazo em MG: 65


In [29]:
# 3) Qual é o municipio da Bahia com mais solicitações deferidas?:
filtro_bahia_municipios_deferidos = (
    (df['uf'] == 'BA') &
    (df['tipo_interessado'] == 'Município') &
    (df['status'] == 'Deferido')
)

# Aplicar o filtro
df_bahia_deferidos = df[filtro_bahia_municipios_deferidos]

# Contar o número de deferimentos por município
municipio_mais_deferimentos = df_bahia_deferidos['interessado'].value_counts().idxmax()
quantidade = df_bahia_deferidos['interessado'].value_counts().max()

print(f'O município da Bahia com mais solicitações deferidas é: {municipio_mais_deferimentos} ({quantidade} solicitações).')

O município da Bahia com mais solicitações deferidas é: Teixeira de Freitas (4 solicitações).


In [31]:
# 4) Salvando um arquivo .csv com os dados da Bahia, com interessado = 'Estado':
filtro = (df["uf"] == "BA") & (df["tipo_interessado"] == "Estado")
df[filtro].to_csv("bahia_estado.csv", index=False)
print("Arquivo salvo como 'bahia_estado.csv'")

Arquivo salvo como 'bahia_estado.csv'
